# Data Preprocessing Notebook

This notebook loads scraped Reddit data, performs cleaning and preprocessing,
and creates a unified dataset for analysis.

## Steps:
1. Load all scraped CSV files
2. Merge into a single DataFrame
3. Text cleaning (lowercase, URL removal)
4. Create full_text column (title + selftext)
5. Mark political posts
6. Export cleaned dataset


In [ ]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from datetime import datetime

# Set project root
PROJECT_ROOT = Path().resolve().parent
DATA_RAW = PROJECT_ROOT / "data" / "raw"
DATA_CLEAN = PROJECT_ROOT / "data" / "clean"

print(f"Project root: {PROJECT_ROOT}")
print(f"Raw data directory: {DATA_RAW}")
print(f"Clean data directory: {DATA_CLEAN}")


## 1. Load All Scraped CSV Files


In [ ]:
# Find all raw CSV files
raw_files = list(DATA_RAW.glob("*.csv"))
print(f"Found {len(raw_files)} raw data files:")
for f in sorted(raw_files):
    print(f"  - {f.name}")


In [ ]:
# Load all raw CSV files into a list of DataFrames
dataframes = []

for file_path in sorted(raw_files):
    # Extract country and phase from filename (e.g., "germany_pre_euro.csv")
    filename = file_path.stem  # Remove .csv extension
    parts = filename.split("_")
    
    if len(parts) >= 2:
        country = parts[0]
        phase = "_".join(parts[1:])  # Handle multi-word phases
        
        try:
            df = pd.read_csv(file_path)
            df["country"] = country
            df["phase"] = phase
            df["source_file"] = file_path.name
            dataframes.append(df)
            print(f"Loaded {len(df)} rows from {file_path.name}")
        except Exception as e:
            print(f"Error loading {file_path.name}: {e}")

print(f"\nTotal files loaded: {len(dataframes)}")


## 2. Merge into Single DataFrame


In [ ]:
# Concatenate all DataFrames
if dataframes:
    df_all = pd.concat(dataframes, ignore_index=True)
    print(f"Merged dataset shape: {df_all.shape}")
    print(f"\nColumns: {list(df_all.columns)}")
    print(f"\nCountries: {df_all['country'].unique()}")
    print(f"\nPhases: {df_all['phase'].unique()}")
    print(f"\nFirst few rows:")
    df_all.head()
else:
    print("No data files found!")


## 3. Text Cleaning Functions

In [ ]:
def remove_urls(text):
    """Remove URLs from text."""
    if pd.isna(text):
        return ""
    # Pattern to match URLs
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\\\(\\\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    return re.sub(url_pattern, '', str(text))

def clean_text(text):
    """
    Clean text: convert to lowercase and remove URLs.
    
    Args:
        text: Input text string
        
    Returns:
        Cleaned text string
    """
    if pd.isna(text):
        return ""
    
    text = str(text)
    text = remove_urls(text)
    text = text.lower()
    
    return text

# Test the cleaning function
test_text = "Check out this link: https://example.com and this is UPPERCASE"
print(f"Original: {test_text}")
print(f"Cleaned: {clean_text(test_text)}")


## 4. Apply Text Cleaning and Create full_text Column


In [ ]:
# Fill NaN values in text columns
df_all["title"] = df_all["title"].fillna("")
df_all["selftext"] = df_all["selftext"].fillna("")

# Clean title and selftext
df_all["title_clean"] = df_all["title"].apply(clean_text)
df_all["selftext_clean"] = df_all["selftext"].apply(clean_text)

# Create full_text column (title + " " + selftext)
df_all["full_text"] = df_all["title_clean"] + " " + df_all["selftext_clean"]
df_all["full_text"] = df_all["full_text"].str.strip()

print(f"Created full_text column")
print(f"Average text length: {df_all['full_text'].str.len().mean():.1f} characters")
print(f"\nSample full_text:")
df_all["full_text"].head(3)


## 5. Mark Political Posts


In [ ]:
# Define political keywords for each country
POLITICAL_KEYWORDS = {
    "germany": [
        "afd", "cdu", "spd", "csu", "gruene", "grüne", "linke",
        "merz", "scholz", "habeck", "migration", "flüchtlinge",
        "asyl", "klima", "heizungsgesetz", "bundestag", "ampel"
    ],
    "netherlands": [
        "vvd", "d66", "pvv", "wilders", "rutte", "klimaat",
        "immigratie", "verkiezingen", "kabinet"
    ],
    "france": [
        "macron", "rn", "mélenchon", "melenchon", "immigration",
        "climat", "gouvernement", "élection", "election", "assemblée", "assemblee"
    ]
}

def is_political(text, country):
    """
    Check if text contains political keywords for the given country.
    
    Args:
        text: Text to check
        country: Country name (germany, netherlands, france)
        
    Returns:
        bool: True if text contains political keywords
    """
    if pd.isna(text) or text == "":
        return False
    
    keywords = POLITICAL_KEYWORDS.get(country, [])
    text_lower = str(text).lower()
    
    return any(keyword.lower() in text_lower for keyword in keywords)

# Mark political posts
df_all["is_political"] = df_all.apply(
    lambda row: is_political(row["full_text"], row["country"]),
    axis=1
)

print(f"Political posts: {df_all['is_political'].sum()} ({df_all['is_political'].mean()*100:.1f}%)")
print(f"\nPolitical posts by country:")
print(df_all.groupby("country")["is_political"].agg(["sum", "mean"]))
print(f"\nPolitical posts by phase:")
print(df_all.groupby("phase")["is_political"].agg(["sum", "mean"]))


## 6. Data Summary and Quality Checks


In [ ]:
# Convert created_utc to datetime if it's a string
if df_all["created_utc"].dtype == "object":
    df_all["created_utc"] = pd.to_datetime(df_all["created_utc"])

# Create date column for easier filtering
df_all["date"] = pd.to_datetime(df_all["created_utc"]).dt.date

# Summary statistics
print("Dataset Summary:")
print(f"Total posts: {len(df_all)}")
print(f"Date range: {df_all['date'].min()} to {df_all['date'].max()}")
print(f"\nPosts by country:")
print(df_all["country"].value_counts())
print(f"\nPosts by phase:")
print(df_all["phase"].value_counts())
print(f"\nPosts by country and phase:")
print(pd.crosstab(df_all["country"], df_all["phase"]))


## 7. Export Cleaned Dataset


In [ ]:
# Select columns for final dataset
columns_to_export = [
    "id", "title", "selftext", "title_clean", "selftext_clean", "full_text",
    "author", "created_utc", "date", "score", "upvote_ratio", "num_comments",
    "url", "permalink", "subreddit", "country", "phase", "is_political",
    "is_self", "over_18", "source_file"
]

# Filter to only include columns that exist
available_columns = [col for col in columns_to_export if col in df_all.columns]
df_clean = df_all[available_columns].copy()

# Save to CSV
output_file = DATA_CLEAN / "all_countries_clean.csv"
df_clean.to_csv(output_file, index=False)

print(f"✓ Saved cleaned dataset to: {output_file}")
print(f"  Shape: {df_clean.shape}")
print(f"  Columns: {list(df_clean.columns)}")
print(f"\nFirst few rows:")
df_clean.head()


## 8. Quick Visualization (Optional)


In [ ]:
# Optional: Quick visualization of data distribution
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Posts by country
df_clean["country"].value_counts().plot(kind="bar", ax=axes[0], color="steelblue")
axes[0].set_title("Posts by Country")
axes[0].set_xlabel("Country")
axes[0].set_ylabel("Number of Posts")
axes[0].tick_params(axis="x", rotation=45)

# Political vs non-political posts
df_clean["is_political"].value_counts().plot(kind="bar", ax=axes[1], color=["lightcoral", "lightgreen"])
axes[1].set_title("Political vs Non-Political Posts")
axes[1].set_xlabel("Is Political")
axes[1].set_ylabel("Number of Posts")
axes[1].set_xticklabels(["Non-Political", "Political"], rotation=0)

plt.tight_layout()
plt.show()

print("\nData preprocessing complete!")
